# Run the fitting routine to assess errors
- Convert to units of eV
- Fit the straggling function

In the first half of this notebook, we use a least squares approaching to fitting the model PDF to the histogram.

In the second half, we use a log-likelihood approach to fitting the model PDF to the histogram.

In [ ]:
# native modules
%matplotlib widget
from collections import defaultdict
import glob
import os
import sys
sys.path.append('/ifs/missions/projects/plcosmic/hst_cosmic_rays/pipeline/utils')

# Local module in the pipeline/utils directory
import fit_energy

# external modules
import astropy.constants as physical_constants
import astropy.units as u
import iminuit
from iminuit.cost import LeastSquares
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import pymc3 as pm
from pylandau import landau, langau
import statsmodels.api as sm
import scipy.ndimage as ndimage
from scipy.optimize import least_squares
from tqdm import tqdm

In [ ]:
data_path = '/ifs/missions/projects/plcosmic/hst_cosmic_rays/pipeline/utils'

In [ ]:
stis_data, stis_ddf = fit_energy.read_ddf(fname=f'{data_path}/stis_ccd_cr_params_2000to2004.txt')
hrc_data, hrc_ddf = fit_energy.read_ddf(fname=f'{data_path}/acs_hrc_cr_params_Apr2002toJan2005.txt')
wfpc2_data, wfpc2_ddf = fit_energy.read_ddf(fname=f'{data_path}/wfpc2_cr_params_2001to2004.txt')
wfc_data, wfc_ddf = fit_energy.read_ddf(fname=f'{data_path}/acs_wfc_cr_params_2002to2005')
uvis_data, uvis_ddf = fit_energy.read_ddf(fname=f'{data_path}/wfc3_uvis_cr_params_2012to2015.txt')

### Data filtering
- Trim the dataset to get the subset of cosmic rays with path lengths between 280 and 300 $\mu$m

In [ ]:
interval = (280*u.micrometer, 300*u.micrometer)
datasets = {'STIS/CCD': None, 'ACS/HRC': None, 'ACS/WFC': None, 'WFPC2': None}

### Comparing the normalized distributions

In [ ]:
for key, data in zip(datasets.keys(), [stis_data, hrc_data, wfc_data, wfpc2_data]):
    path_length = data['track_path_length']
    energy_deposited = data['energy_deposited']
    
    path_cut = (path_length >= interval[0]) & (path_length < interval[1])
    path_data = path_length[path_cut]
    energy_data = energy_deposited[path_cut]
    energy_data = energy_data[energy_data <= 500*u.keV]
    datasets[key] = {'energy_deposited': energy_data, 'path_length': path_data}

In [ ]:
nbins = 60
hrange = (0,300)
for key, values in datasets.items():
    dhist, dedges = np.histogram(values['energy_deposited'], bins=nbins, range=hrange, density=True)
    datasets[key]['dhist'] = dhist
    datasets[key]['centers'] = 0.5*(dedges[:-1] + dedges[1:])


In [ ]:
shift=0
fig, ax = plt.subplots(nrows=1, ncols=1)
for key, values in datasets.items():
    if key == 'WFPC2':
        ax.step(values['centers']+shift*u.keV, values['dhist'], label=key)
    else:
        ax.step(values['centers'], values['dhist'], label=key)
ax.legend(loc='upper right')

In [ ]:
stis_fit_results = [61.1018, 12.1227, 4.08194, 0.0138702]
hrc_fit_results = [60.8783, 7.98247, 10.3883, 0.017024]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(datasets['STIS/CCD']['centers'], datasets['STIS/CCD']['dhist'], label='STIS', c='tab:red')
ax.plot(datasets['STIS/CCD']['centers'], langau(datasets['STIS/CCD']['centers'], *stis_fit_results), c='tab:red')
ax.step(datasets['ACS/HRC']['centers'], datasets['ACS/HRC']['dhist'], label='HRC', c='tab:purple')
ax.plot(datasets['ACS/HRC']['centers'], langau(datasets['ACS/HRC']['centers'], *hrc_fit_results), c='tab:purple')
ax.legend(loc='best', edgecolor='k')

##### Generate the STIS data

In [ ]:
path_length = stis_data['track_path_length']
energy_deposited = stis_data['energy_deposited']

path_cut = (path_length >= interval[0]) & (path_length < interval[1])
path_data = path_length[path_cut]
energy_data = energy_deposited[path_cut]
energy_data = energy_data[energy_data <= 500*u.keV]
print(f"Number of CRs: {len(energy_data):,}")

##### Generate the HRC data

In [ ]:
interval = (280*u.micrometer, 300*u.micrometer)
hrc_path_length = hrc_data['track_path_length']
hrc_energy_deposited = hrc_data['energy_deposited']

hrc_path_cut = (hrc_path_length >= interval[0]) & (hrc_path_length < interval[1])
hrc_path_data = hrc_path_length[hrc_path_cut]
hrc_energy_data = hrc_energy_deposited[hrc_path_cut]
hrc_energy_data = hrc_energy_data[hrc_energy_data <= 500*u.keV]
print(f"Number of CRs: {len(hrc_energy_data):,}")

In [ ]:
nbins = 60
hrange = (0,300)

In [ ]:
hist, edges = np.histogram(energy_data, bins=nbins, range=hrange, density=False)
dhist, dedges = np.histogram(energy_data, bins=nbins, range=hrange, density=True)

In [ ]:
hrc_hist, hrc_edges = np.histogram(hrc_energy_data, bins=nbins, range=hrange, density=False)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(edges[1:], hist, color='tab:blue', label='right edge')
ax.step(0.5*(edges[:-1] + edges[1:]), hist, color='tab:green', label='center')
# ax.plot(kde.support, kde.density, c='k', ls='--')
ax.legend()

In [ ]:
left_edges = edges[:-1]
right_edges = edges[1:]
centers = 0.5*(left_edges + right_edges)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(centers, hist, color='tab:green', label='STIS/CCD')
ax.step(centers, hrc_hist, color='tab:purple', label='ACS/HRC')
ax.legend(loc='upper right')

In [ ]:
np.percentile(energy_data, q=[25,75])

In [ ]:
hrc_dhist, dedges = np.histogram(hrc_energy_data, bins=nbins, range=hrange, density=True)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(dedges[1:], dhist, color='tab:blue', label='right edge')
ax.step(0.5*(dedges[:-1] + dedges[1:]), dhist, color='tab:green', label='center')
# ax.plot(kde.support, kde.density, c='k', ls='--')
ax.legend()
ax.set_title('Normalized Histogram PDF')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(centers, dhist, color='tab:green', label='STIS/CCD')
ax.step(centers, hrc_dhist, color='tab:purple', label='ACS/HRC')
ax.legend(loc='upper right')

Compute the scaling factor between the normalized and unnormalized histograms

In [ ]:
scaling_factor = hist[hist.argmax()]/dhist[dhist.argmax()]

Configure the settings to be used by `statsmodels` to perform the kernel density estimation

In [ ]:
estimator_settings = sm.nonparametric.EstimatorSettings(
        efficient=True, randomize=True, 
        n_res=1000, n_sub=300, return_median=True, 
        return_only_bw=False, n_jobs=-1
    )
ml_bw = sm.nonparametric.KDEMultivariate(
            energy_data, 
            'c', 
            bw = 'cv_ml', 
            defaults=estimator_settings)

In [ ]:
ml_bw

In [ ]:
ml_bw.imse(ml_bw.bw)

In [ ]:
len(energy_data)

In [ ]:
kde = sm.nonparametric.KDEUnivariate(energy_data)
kde.fit(bw=ml_bw, kernel='gau', cut=0)

### Estimating the FWHM of the KDE

In [ ]:
max_point = kde.density.argmax()
max_pdf = kde.density[max_point]
max_val = kde.support[max_point]
print(max_val, max_pdf)

In [ ]:
half_max = max_pdf/2
# find the values with 1e-3 of half the maximum
close_vals = np.isclose(half_max, kde.density, atol=1e-5)

In [ ]:
close_xvals = kde.support[close_vals]
close_pdfvals = kde.density[close_vals]

In [ ]:
close_xvals

### Estimating the error in the KDE

In [ ]:
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import sproot

In [ ]:
kde_spline = UnivariateSpline(kde.support, kde.density, k=3, s=0) # cubic spline
# second_derivative = kde_spline.derivative(n=2)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1)
ax[0].plot(kde.support, kde.density)
ax[0].plot(kde.support+20, kde_spline(kde.support))
ax[1].plot(kde.support, second_derivative(kde.support))

In [ ]:
def compute_mse(kde):
    x = kde.support
    y = kde.density
    
    # Fit a cubic spline to the data
    kde_spline = UnivariateSpline(kde.support, kde.density, k=3, s=0)
    
    # Compute the second derivative of the spline using the central difference method
    second_derivative = kde_spline.derivative(n=2)
    
    # Compute the variance of the KDE
    

In [ ]:
len(kde.density), len(kde.support)

In [ ]:
0.5*(edges[:-1] + edges[1:])

Use the kde generated above to compute the expected value in 0.5 step increments starting at 1 keV.

Since the kde is an estimation of the underlying probability density distribution for the dataset, it is normalized such that the integral of the entire domain is 1. Hence we need to compare it to the normalized histogram

In [ ]:
xvals = np.linspace(1, 300, 599)
y_obs = kde.evaluate(xvals)

In [ ]:
fwhm = close_xvals[-1] - close_xvals[0]

In [ ]:
fwhm/12.1

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(centers, dhist)
ax.plot(xvals, y_obs, c='k', ls='--')
ax.set_xlim(0,300)
ax.axvline(close_xvals[0])
ax.axvline(close_xvals[-1])

Setting up initial guess for each of the parameters in the fit

Estimating $\xi$

In [ ]:
# Compute an esimtate of xi
z=1
Z=14
A=28.0855*u.g/u.mol
v = 0.98*physical_constants.c
c = physical_constants.c
K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
m_e = physical_constants.m_e
beta_val = v/c
estimated_path = 290 * u.micrometer
si_density = 2.329 * u.g/u.cm**3
xi_guess = K/2 * Z/A * z**2 *  1/(beta_val)**2 * si_density *estimated_path
xi_guess = xi_guess.to('keV')
xi_guess = xi_guess.value
limit_xi = (0.8*xi_guess, 3.5*xi_guess)
print(f"xi: {xi_guess:.3f}")
print(f"xi range: ({limit_xi[0]:.3f}, {limit_xi[1]:.3f})")

Estimating $\sigma$

In [ ]:
sigma_guess = 5
limit_sigma = (0.5*sigma_guess, 3*sigma_guess)

Estimating mpv

In [ ]:
mpv_guess = centers[hist.argmax()].value
limit_mpv = (0.85*mpv_guess, 1.2*mpv_guess)

Estimating the maximum probability

In [ ]:
A_guess = hist.max()
limit_A = (0.5*hist.max(), 2*hist.max())

In [ ]:
for param, param_range in zip([xi_guess, sigma_guess, mpv_guess],[limit_xi, limit_sigma, limit_mpv]):
    print(f"{param:.3f}")
    print(f"({param_range[0]:.3f}, {param_range[1]:.3f})\n")

In [ ]:
# initial guess for parameters
p0 = [mpv_guess, xi_guess, sigma_guess, A_guess]

In [ ]:
def fit_landau_leastsq(
    x, 
    y_obs, 
    p0, 
    limit_mpv=None,
    limit_xi=None,
    limit_sigma=None, 
    limit_A=None
):
    least_squares = LeastSquares(
        x=x,
        y=y_obs,
        yerror=np.sqrt(y_obs), # assume poisson errors
        model=landau,
        loss='soft_l1'
    )
    m = iminuit.Minuit(
        least_squares,
        mpv=p0[0],
        limit_mpv=limit_mpv,
        eta=p0[1],
        limit_eta=limit_xi,
        A=p0[2],
        limit_A=limit_A,
        pedantic=False
    )
    
    # Minimize the gradient
    m.migrad()
    
    # Compute hessian errors
    m.hesse()
    try:
        assert m.valid
    except AssertionError:
        pass
    else:
        # Compute Minos errors
        m.minos()
    return m

In [ ]:
def fit_langau_leastsq(
    x, 
    y_obs, 
    p0, 
    limit_mpv=None,
    limit_xi=None,
    limit_sigma=None, 
    limit_A=None
):
    least_squares = LeastSquares(
        x=x,
        y=y_obs,
        yerror=np.sqrt(y_obs), # assume poisson errors
        model=langau,
        loss='soft_l1'
    )
    m = iminuit.Minuit(
        least_squares,
        errordef=1,
        mpv=p0[0],
        limit_mpv=limit_mpv,
        eta=p0[1],
        limit_eta=limit_xi,
        sigma=p0[2],
        limit_sigma=limit_sigma,
        A=p0[2],
        limit_A=limit_A,
        pedantic=True,
        scale_langau=True,
        fix_scale_langau=True
    )
    m.migrad()
    
    if not m.valid:
        raise RuntimeError('Fit did not converge')
    
    # Compute minos errors
    m.minos()
    
    fitted_params = m.values.values()
    param_errors = m.merrors
    param_errors = np.array(
        [(param_errors['mpv'].lower, param_errors['mpv'].upper),
         (param_errors['eta'].lower, param_errors['eta'].upper),
         (param_errors['sigma'].lower, param_errors['sigma'].upper),
         (param_errors['A'].lower, param_errors['A'].upper)]
    )
    for val, err in zip(fitted_params, param_errors):
        print(f"{val+err[0]:.3f} < {val:0.3f} < {val+err[1]:0.3f}")
    return m

In [ ]:
landau_fit_obj = fit_landau_leastsq(
    centers[hist>0], 
    hist[hist>0], 
    p0=[p0[0], p0[1], p0[-1]],
    limit_xi=limit_xi,
    limit_mpv=None,
    limit_A=None
)

In [ ]:
landau_fit_obj.migrad()

In [ ]:
xi_guess

In [ ]:
landau_fit_obj.values.values()

In [ ]:
landau_fit_obj_dhist = fit_landau_leastsq(
    centers[dhist>0], 
    1e5*dhist[dhist>0].value, 
    p0=[p0[0], p0[1], p0[-1]],
    limit_xi=limit_xi,
    limit_mpv=None,
    limit_A=None
)

In [ ]:
landau_fit_obj_dhist.migrad()

In [ ]:
landau_fit_obj_dhist.minos()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(centers, landau(centers, *landau_fit_obj_dhist.values.values())/1e5)
ax.step(centers, dhist, color='tab:green')
ax.plot(xvals, y_obs, c='k', ls='--')

In [ ]:
langau_fit_obj = fit_langau_leastsq(
    centers[dhist>0], 
    1e5*dhist[dhist>0].value, 
    p0=p0,
    limit_xi=limit_xi,
    limit_mpv=limit_mpv,
    limit_sigma=limit_sigma,
    limit_A=limit_A
)

In [ ]:
langau_fit_obj.minos()

In [ ]:
langau_fit_obj.merrors

In [ ]:
langau_fit_obj.values

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(centers, langau(centers, *langau_fit_obj.values.values())/1e5)
ax.step(centers, dhist, color='tab:green')
ax.plot(xvals, y_obs, c='k', ls='--')

In [ ]:
langau_fit_obj = fit_langau_leastsq(
    centers[hist>0], 
    hist[hist>0], 
    p0=p0,
    limit_xi=limit_xi,
    limit_mpv=limit_mpv,
    limit_sigma=limit_sigma,
    limit_A=limit_A
)

In [ ]:
langau_fit_obj.minos()

In [ ]:
langau_fit_obj.merrors

In [ ]:
langau_fit_obj.values.values()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(centers, langau(centers, *langau_fit_obj.values.values()))
ax.plot(centers, landau(centers, *landau_fit_obj.values.values()), c='k', ls='--')
ax.step(centers, hist, color='tab:green')

Now we repeat the same fitting process, but this time we use the KDE-derived PDF

In [ ]:
langau_fit_obj_kde = fit_langau_leastsq(
    xvals, 
    1e5*y_obs, 
    p0=p0,
    limit_xi=(1, None),
    limit_mpv=(0,None),
    limit_sigma=(0, None),
    limit_A=(0, None)
)

In [ ]:
langau_fit_obj_kde.minos?

In [ ]:
langau_fit_obj_kde.minos(sigma=3)

In [ ]:
langau_fit_obj_kde.minos()

In [ ]:
langau_fit_obj_kde.values

In [ ]:
4 * langau_fit_obj_kde.values['eta']

In [ ]:
y_fit = langau(xvals, *langau_fit_obj_kde.values.values())

In [ ]:
residuals = y_obs - y_fit/1e5

In [ ]:
resid_hist, resid_edges = np.histogram(residuals, range=(-0.0005, 0.0005), bins=50)
resid_centers = 0.5*(resid_edges[:-1] + resid_edges[1:])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(resid_centers, resid_hist)

In [ ]:
rms_err = np.sqrt(np.mean(np.square(1e5*y_obs - y_fit)))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
# ax.plot(xvals, langau(xvals, *langau_fit_obj_kde.values.values()))
ax.errorbar(xvals[::3], y_fit[::3], yerr=rms_err, fmt='o', ms=2)
ax.plot(xvals, 1e5*y_obs, c='k', ls='--')

In [ ]:
rms_err

In [ ]:
resid

In [ ]:
landau_fit_obj_kde = fit_landau_leastsq(
    xvals, 
    1e5*y_obs, 
    p0=[p0[0], p0[1], p0[-1]],
    limit_xi=(1.5,None),
    limit_mpv=None,
    limit_A=None
)

In [ ]:
landau_fit_obj_kde.minos()

In [ ]:
landau_fit_obj_kde.values

In [ ]:
landau_fit_obj_kde.values['A'] /=1e5

In [ ]:
landau_fit_obj_kde.values['A']

In [ ]:
langau_fit_obj_kde.values['A'] /= 1e5

In [ ]:
errors = np.asarray([(langau_fit_obj_kde.merrors[key].lower,langau_fit_obj_kde.merrors[key].upper)  for key in langau_fit_obj_kde.merrors.keys()])

In [ ]:
errors[-1] = errors[-1]/1e5

In [ ]:
errors

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(xvals, langau(xvals, *(langau_fit_obj_kde.values.values()[:4] + errors[:,0])), c='tab:purple',ls='--')
ax.plot(xvals, langau(xvals, *(langau_fit_obj_kde.values.values()[:4] + errors[:,1])), c='tab:purple',ls='--')
ax.plot(xvals, langau(xvals, *langau_fit_obj_kde.values.values()), c='tab:red')
# ax.plot(xvals, landau(xvals, *landau_fit_obj_kde.values.values()), c='tab:blue')
ax.plot(xvals, y_obs, color='tab:green')

In [ ]:
def compute_landau_mpv(m, v, z, Z, A, I, dx):
    c = physical_constants.c
    K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
    m_e = physical_constants.m_e
    constant = K/2 * Z/A * z**2 * dx * 1/(v/c)**2
#     density_corr = sternheimer_density_corr(v/c, gamma(v))
    density_corr = 0
    scaling = np.log((2 * m_e * c**2 * (v/c)**2 * gamma(v)**2)/I) + np.log(constant/I) + 0.2 - (v/c)**2 - density_corr 
    return constant * scaling

### Compute Landau + Gauss for particle with $\beta$=0.98

- Reproducing the curves in Bichsel 1988
    - $\beta\gamma$ = 2.1 --> $\beta=0.9$
    - $\beta\gamma$ = 8.5 --> $\beta=0.99315$

In [ ]:
beta_gamma = 2.1
beta_gamma = 8.5

In [ ]:
beta = 0.9
gamma = np.sqrt(1/(1-beta**2))

In [ ]:
beta*gamma

In [ ]:
# Compute an esimtate of xi
z=1
Z=14
A=28.0855*u.g/u.mol
c = physical_constants.c
K = 0.307075 * u.megaelectronvolt * u.cm**2 / u.mol
m_e = physical_constants.m_e
estimated_path = 290 * u.micrometer
si_density = 2.329 * u.g/u.cm**3
xi = K/2 * Z/A * z**2 *  1/(beta)**2 * si_density *estimated_path
xi = xi.to('keV')


In [ ]:
xi

In [ ]:
langau?

In [ ]:
x = np.linspace(1,1000,10000)

In [ ]:
y = langau(x, mpv=82.46, eta=xi.value, sigma=3.2, A=1)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(x, y)

In [ ]:
ymax = y[y.argmax()]
half_ymax = ymax / 2

In [ ]:
half_ymax

In [ ]:
flag = np.isclose(half_ymax, y, atol=2e-2)

In [ ]:
flag.sum()

In [ ]:
xpoints = x[flag]

In [ ]:
xpoints[-1] - xpoints[0]

In [ ]:
flag.sum()

In [ ]:
def estimate_fwhm(x, y):
    
    ymax = y[y.argmax()]
    half_ymax = ymax / 2
    
    # find all values within 1/1000 of half_ymax
    flag = [False]
    tol = 1e-4
    while sum(flag) < 4:
        flag = np.isclose(half_ymax, y, atol=tol)
        tol += 0.0005
    xpoints = x[flag]
    print(xpoints)
    fwhm = xpoints[-1] - xpoints[0]
    return fwhm

In [ ]:
fwhm = estimate_fwhm(x, y)

In [ ]:
fwhm

## Fitting the distributions of track lengths

In [ ]:
stis_data, stis_ddf = fit_energy.read_ddf(fname=f'{data_path}/stis_ccd_cr_params_2000to2004.txt')

In [ ]:
wfc_data, wfc_ddf = fit_energy.read_ddf(fname=f'{data_path}/acs_wfc_cr_params_2002to2005')

In [ ]:
import dask.array as da

In [ ]:
full_dset = None

In [ ]:
full_dset = []
for i, data in enumerate([stis_ddf, wfc_ddf, wfpc2_ddf, hrc_ddf, uvis_ddf]):
    hist, edges = np.histogram(data['track_path_length'], bins=40, range=(60, 1000), density=True)
    if i == 0:
        avg_hist = hist
    else:
        avg_hist += hist
    full_dset.append(da.from_array(data['track_path_length'].compute()))
avg_hist /= 5

In [ ]:
full_dset = da.concatenate(full_dset, axis=0)

In [ ]:
full_dset.shape

In [ ]:
total_hist, total_edges = np.histogram(full_dset, range=(60, 1000), bins=40, density=True)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(edges[1:], avg_hist, c='k')
ax.step(total_edges[1:], total_hist)
ax.set_yscale('log')

In [ ]:
f"{3825790.1825247086:.2e}"

In [ ]:
centers = 0.5*(edges[1:] + edges[:-1])

In [ ]:
def track_length_pdf_shielding(x, A, n):
    return  A/(x**(n+2))

In [ ]:
least_squares = LeastSquares(
        x=centers[1:],
        y=1e7*total_hist[1:],
        yerror=np.sqrt(1e7*total_hist)[1:], # assume poisson errors
        model=track_length_pdf_shielding,
        loss='soft_l1'
    )

In [ ]:
m = iminuit.Minuit(
        least_squares,
        errordef=1,
        A=1e7*total_hist.max(),
        n=2,
        limit_n=[1,4]
)
m.migrad()

In [ ]:
m.migrad()

In [ ]:
values = m.values.values()

In [ ]:
values

In [ ]:
values[0] /= 1e7

In [ ]:
values

In [ ]:
f = lambda x: 1/x**2

In [ ]:
y = f(centers)

In [ ]:
y

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.step(centers, hist)
ax.set_yscale('log')
ax.plot(centers, track_length_pdf_shielding(centers, values[0], values[1]))